In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
import cv2

import numpy as np

import glob




width=200

height=200




Data=[]

Target=[]



for k in range(0,8):

  path=r'/content/drive/My Drive/DataSet/Tomate Pest/' + str(k) + '/*.jpg'

  file_list = glob.glob(path)

  path=r'/content/drive/My Drive/DataSet/Tomate Pest/' + str(k) + '/*.JPG'

  file_list_1 = glob.glob(path)

  for i in file_list:
    im=cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im=cv2.resize(im,(width,height))
    Data.append(im)
    Target.append(k)

  for i in file_list_1:
    im=cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im=cv2.resize(im,(width,height))

  Data.append(im)
  Target.append(k)

In [12]:
from tensorflow.keras.utils import to_categorical
Final_Target = to_categorical(Target)

In [13]:
Final_Data=np.reshape(Data,(len(Data),height,width,1))

In [18]:
# CNN LSTM
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model

myInput=layers.Input(shape=(height,width,1))
conv1=SeparableConv2D(128,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
conv2=SeparableConv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
Time1=layers.TimeDistributed(Flatten())(max2)
lstm1=LSTM(150)(Time1)
des1 =Dense(50, activation='relu')(lstm1)
Extract_Model = Model(myInput, des1)
des2 =Dense(20, activation='relu')(des1)
out_layer =Dense(8, activation='softmax')(des2)
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

verbose, epochs, batch_size = 1, 10, 32
model.fit(Final_Data,Final_Target, epochs=epochs, batch_size=batch_size, verbose=verbose)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200, 200, 1)]     0         
                                                                 
 separable_conv2d_6 (Separab  (None, 200, 200, 128)    265       
 leConv2D)                                                       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 100, 100, 128)    0         
 2D)                                                             
                                                                 
 separable_conv2d_7 (Separab  (None, 100, 100, 64)     9408      
 leConv2D)                                                       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 50, 50, 64)       0         
 2D)                                                       

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, Target, test_size=0.3, random_state=42)

In [20]:
from tensorflow.keras.utils import to_categorical
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

In [21]:
trainX=np.reshape(X_train,(len(X_train),height,width,1))
testX=np.reshape(X_test,(len(X_test),height,width,1))

In [22]:
trainFeature=(Extract_Model.predict(trainX))
testFeature=(Extract_Model.predict(testX))

37/37 [==============================] - 46s 1s/step


In [23]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [24]:
from sklearn import tree
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature, y_train)

DecisionTreeClassifier()

In [25]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature, y_train)

KNeighborsClassifier(n_neighbors=3)

In [26]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature, y_train)
Class_Voting=classifier_Voting.predict(testFeature)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,Class_Voting))

In [28]:
print('Accuracy= ' + str(round(accuracy_score(y_test, Class_Voting)*100)))

Accuracy= 94
